## Machine Translation

`Task:`
- Develop a model that can translate a Tamil input sentence to English. 
- Analyze the impact of various embedding techniques in the translation task. 
- Use  sequence  model  to  translate  the  Tamil  input  sentence  and  analyze  the  performance  of various techniques. 
- Translate the input sentences using Transformer model and understand its functioning.  
- Translate the following Tamil Phrase and evaluate your model based on the expected output.  
    - Input 1: நான் மிகவும் சந்த ாஷமாக இருக்கிதேன்  
    - Expected Output: Im so happy  
    - Input 2: அது அவசியமில்லை 
    - Expected Output: It wasnt necessary 
    - Input 3:  யவுசசய்து அல  மீண்டும் சசய்யவும் 
    - Expected Output: Please do that again 
    - Input 4: அது ஒரு நல்ை தயாசலை 
    - Expected Output: That is a good idea 
    - Input 5: அவர்கள் ஒன்ோக தவலை சசய்ய ஒப்புக்சகாண்டைர் 
    - Expected Output: They agreed to work together 

### Import the libraries

In [1]:
import keras
import string
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.layers import LSTM,Embedding,RepeatVector,Dense,Input

### Load the data and prepare the data

In [2]:
def read_sentences(path):
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    return lines

In [3]:
en_path = 'Dataset/data.en'
ta_path = 'Dataset/data.ta'

In [4]:
en_sentences, ta_sentences = [], []

In [5]:
for i in range(1,7):
    en_sentences.extend(read_sentences(en_path + str(i)))
    ta_sentences.extend(read_sentences(ta_path + str(i)))

In [6]:
en_sentences[:2], ta_sentences[:2]

(['moreover all the vessels , which king ahaz in his reign did cast away in his transgression , have we prepared and sanctified , and , behold , they are before the altar of the lord .',
  'similar conditions will be imposed if the sri lankan government is given an imf loan .'],
 ['ராஜாவாகிய ஆகாஸ் அரசாளும்போது தம்முடைய பாதகத்தினால் எறிந்துபோட்ட சகல பணிமுட்டுகளையும் முஸ்திப்பாக்கிப் பரிசுத்தம்பண்ணினோம்; இதோ , அவைகள் கர்த்தரின் ஆலயத்திற்கு முன்பாக இருக்கிறது என்றார்கள் .',
  'சர்வதேச நாணய நிதியம் இலங்கைக்கு கடன் வழங்கினால் இதே போன்ற நிபந்தனைகள் திணிக்கப்படும் .'])

In [7]:
len(en_sentences), len(ta_sentences)

(289456, 289456)

In [8]:
with open('tam.txt','r',encoding='utf-8') as f:
    lines = f.read().split('\n')

In [9]:
lines[:2]

['I slept.\tநான் தூங்கினேன்.\tCC-BY 2.0 (France) Attribution: tatoeba.org #3199633 (CM) & #7098307 (Singaravelu)',
 'Calm down.\tஅமைதியாக இருங்கள்\tCC-BY 2.0 (France) Attribution: tatoeba.org #435575 (CK) & #4268041 (Singaravelu)']

In [10]:
lines[0].split('\t')

['I slept.',
 'நான் தூங்கினேன்.',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #3199633 (CM) & #7098307 (Singaravelu)']

In [11]:
for line in lines[:-1]:
    en_sent, ta_sent, _ = line.split('\t')
    en_sentences.append(en_sent)
    ta_sentences.append(ta_sent)

In [12]:
len(en_sentences), len(ta_sentences)

(289657, 289657)

In [13]:
# taking only the first 10000 sentences
en_sentences = en_sentences[:10000]
ta_sentences = ta_sentences[:10000]

In [14]:
english_sentences =['START_ '+sent.lower().translate(str.maketrans('','',string.punctuation))+' _END' for sent in en_sentences] # for each sentence add start and end token, lower case and remove punctuation - str.maketrans() method is creating a translation table that maps every punctuation character to None
tamil_sentences =[sent.translate(str.maketrans('','',string.punctuation))for sent in ta_sentences] # remove punctuation

In [15]:
english_sentences[0], tamil_sentences[0]

('START_ moreover all the vessels  which king ahaz in his reign did cast away in his transgression  have we prepared and sanctified  and  behold  they are before the altar of the lord  _END',
 'ராஜாவாகிய ஆகாஸ் அரசாளும்போது தம்முடைய பாதகத்தினால் எறிந்துபோட்ட சகல பணிமுட்டுகளையும் முஸ்திப்பாக்கிப் பரிசுத்தம்பண்ணினோம் இதோ  அவைகள் கர்த்தரின் ஆலயத்திற்கு முன்பாக இருக்கிறது என்றார்கள் ')

In [16]:
lines = pd.DataFrame({'english':english_sentences,'tamil':tamil_sentences})
lines.head()

,english,tamil
0,START_ moreover all the vessels which king ah...,ராஜாவாகிய ஆகாஸ் அரசாளும்போது தம்முடைய பாதகத்தி...
1,START_ similar conditions will be imposed if t...,சர்வதேச நாணய நிதியம் இலங்கைக்கு கடன் வழங்கினால...
2,START_ now kornelius argues the opposite inste...,தற்போது அதற்கு எதிராக வாதாடுகிறார் சர்வதேச சட...
3,START_ chrysler the third largest us automaker...,அமெரிக்காவின் மூன்றாம் பெரிய கார் தயாரிப்பு நி...
4,START_ moreover khan has been in exile in ira...,மேலும் இனைவிட்டு தலிபானால் வெளியேற்றப்பட்ட 199...


In [17]:
# split the sentence by space and count the number of words
lines['length_en_sent'] = lines['english'].apply(lambda x:len(x.split(" "))) 
lines['length_ta_sent'] = lines['tamil'].apply(lambda x:len(x.split(" ")))

In [18]:
lines.head()

,english,tamil,length_en_sent,length_ta_sent
0,START_ moreover all the vessels which king ah...,ராஜாவாகிய ஆகாஸ் அரசாளும்போது தம்முடைய பாதகத்தி...,39,19
1,START_ similar conditions will be imposed if t...,சர்வதேச நாணய நிதியம் இலங்கைக்கு கடன் வழங்கினால...,18,11
2,START_ now kornelius argues the opposite inste...,தற்போது அதற்கு எதிராக வாதாடுகிறார் சர்வதேச சட...,27,16
3,START_ chrysler the third largest us automaker...,அமெரிக்காவின் மூன்றாம் பெரிய கார் தயாரிப்பு நி...,33,29
4,START_ moreover khan has been in exile in ira...,மேலும் இனைவிட்டு தலிபானால் வெளியேற்றப்பட்ட 199...,28,11


In [19]:
lines.shape

(10000, 4)

In [20]:
# consider only those sentences with words less than 50
lines = lines[lines['length_en_sent']<=50]
lines = lines[lines['length_ta_sent']<=50]

In [21]:
lines.shape

(9727, 4)

In [22]:
english_sentences[0]

'START_ moreover all the vessels  which king ahaz in his reign did cast away in his transgression  have we prepared and sanctified  and  behold  they are before the altar of the lord  _END'

In [28]:
en_corpus = set()
ta_corpus = set()   

for sent in english_sentences:
    for word in sent.split():
        if word not in en_corpus: en_corpus.add(word)
        
for sent in tamil_sentences:
    for word in sent.split():
        if word not in ta_corpus: ta_corpus.add(word)
        
len(en_corpus), len(ta_corpus)

(18682, 46735)

In [29]:
en_corpus = sorted(list(en_corpus))
ta_corpus = sorted(list(ta_corpus))

In [30]:
num_encoder_tokens = len(ta_corpus)
num_decoder_tokens = len(en_corpus)

In [31]:
num_encoder_tokens, num_decoder_tokens

(46735, 18682)

In [32]:
num_decoder_tokens += 1 # for zero padding

In [33]:
input_token_index = dict([(word,i+1) for i,word in enumerate(en_corpus)])
target_token_index = dict([(word,i+1) for i,word in enumerate(ta_corpus)])

In [34]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(lines['tamil'],lines['english'],test_size = 0.1,random_state=42)

In [ ]:
x_train.to_pickle('x_train.pkl') 
x_test.to_pickle('x_test.pkl')